In [ ]:
!pip install gradio

In [ ]:
!pip install reportlab

In [ ]:
!pip install -U torch torchvision torchaudio
!pip install git+https://github.com/facebookresearch/detectron2.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install timm pycocotools

In [ ]:
import gradio as gr
import cv2
import numpy as np
from PIL import Image
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as ReportLabImage, Table
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
import tempfile
import datetime

# Configure Detectron2 model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Single class for damage detection
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Lower confidence threshold

# Set metadata
MetadataCatalog.get("car_dataset").thing_classes = ["damage"]
metadata = MetadataCatalog.get("car_dataset")

# Initialize predictor
predictor = DefaultPredictor(cfg)

# Repair cost parameters (modify according to your needs)
SEVERITY_LEVELS = {
    'Low': (1, 500),
    'Moderate': (501, 1500),
    'Severe': (1501, 3000)
}

def calculate_severity(num_damages):
    if num_damages == 0:
        return "No Damage", (0, 0)
    elif num_damages <= 2:
        return "Low", SEVERITY_LEVELS['Low']
    elif 3 <= num_damages <= 4:
        return "Moderate", SEVERITY_LEVELS['Moderate']
    else:
        return "Severe", SEVERITY_LEVELS['Severe']

def generate_pdf(image_path, severity, cost_estimate, damage_count):
    # Create a temporary PDF file
    pdf_file = tempfile.NamedTemporaryFile(suffix=".pdf", delete=False)

    doc = SimpleDocTemplate(pdf_file.name, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Title
    title_style = ParagraphStyle(
        'Title',
        parent=styles['Heading1'],
        alignment=1,
        spaceAfter=14
    )
    story.append(Paragraph("Car Damage Assessment Report", title_style))

    # Company Info
    story.append(Paragraph("CarDamage Analyzer Pro", styles['Normal']))
    story.append(Paragraph("Contact: support@cardamage.ai", styles['Normal']))
    story.append(Spacer(1, 12))

    # Add current date
    story.append(Paragraph(f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}", styles['Normal']))
    story.append(Spacer(1, 24))

    # Add damage visualization image
    img = ReportLabImage(image_path, width=4*inch, height=3*inch)
    story.append(img)
    story.append(Spacer(1, 24))

    # Damage Details
    story.append(Paragraph("Damage Assessment Summary:", styles['Heading2']))
    data = [
        ['Severity Level:', severity],
        ['Estimated Repair Cost:', cost_estimate],
        ['Number of Damages:', damage_count]
    ]
    table = Table(data, colWidths=[2*inch, 4*inch])
    table.setStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
        ('GRID', (0,0), (-1,-1), 1, colors.black)
    ])
    story.append(table)
    story.append(Spacer(1, 24))

    # Repair Steps
    repair_steps = {
        'Low': ["1. Minor surface cleaning", "2. Touch-up painting", "3. Protective coating application"],
        'Moderate': ["1. Panel straightening", "2. Medium-level repainting", "3. Component alignment"],
        'Severe': ["1. Structural frame inspection", "2. Component replacement", "3. Full repainting", "4. Mechanical system check"]
    }

    story.append(Paragraph("Recommended Repair Steps:", styles['Heading2']))
    for step in repair_steps.get(severity, ["No repairs needed"]):
        story.append(Paragraph(f"• {step}", styles['Normal']))

    story.append(Spacer(1, 24))
    story.append(Paragraph("Notes:", styles['Heading2']))
    story.append(Paragraph("• Costs are estimates based on average market rates", styles['Normal']))
    story.append(Paragraph("• Final cost may vary based on actual damage extent", styles['Normal']))

    doc.build(story)
    return pdf_file.name

def predict_damage(image):
    # Convert PIL Image to OpenCV format
    img = np.array(image)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Run inference
    outputs = predictor(img_bgr)
    instances = outputs["instances"].to("cpu")

    # Process detections
    num_damages = len(instances)

    # Calculate severity and cost
    severity_level, cost_range = calculate_severity(num_damages)
    cost_estimate = f"₹{cost_range[0]} - ₹{cost_range[1]}" if num_damages > 0 else "No damage detected"

    # Visualize results
    v = Visualizer(img_bgr[:, :, ::-1], metadata=metadata, scale=0.8)
    out = v.draw_instance_predictions(instances)
    result_img = Image.fromarray(out.get_image()[:, :, ::-1])

    # Generate PDF report
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_img:
        result_img.save(temp_img.name, format='JPEG')
        pdf_path = generate_pdf(temp_img.name, severity_level, cost_estimate, str(num_damages))

    return result_img, severity_level, cost_estimate, str(num_damages), pdf_path

# Create Gradio interface
inputs = gr.Image(label="Upload Car Image", type="pil")
outputs = [
    gr.Image(label="Damage Visualization"),
    gr.Textbox(label="Damage Severity"),
    gr.Textbox(label="Estimated Repair Cost"),
    gr.Textbox(label="Damage Count"),
    gr.File(label="Download Report")
]

title = "Car Damage Assessment System"
description = "Upload an image of a damaged car to assess severity, repair cost, and view detected damages"

examples = [["sample_damage1.jpg"], ["sample_damage2.jpg"]]  # Add your sample images

gr.Interface(
    fn=predict_damage,
    inputs=inputs,
    outputs=outputs,
    title=title,
    description=description,
    examples=examples
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://01bdbc3a983a946cbb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio opencv-python numpy pillow torch torchvision reportlab

In [ ]:
import gradio as gr
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import tempfile
import datetime
import random
import os
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as ReportLabImage, Table
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors

# Configure model settings
try:
    from detectron2.config import get_cfg
    from detectron2.engine import DefaultPredictor
    from detectron2 import model_zoo
    from detectron2.utils.visualizer import Visualizer
    from detectron2.data import MetadataCatalog

    # Configure Detectron2 model
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_final.pth"
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Single class for damage detection
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Lower confidence threshold

    # Set metadata
    MetadataCatalog.get("car_dataset").thing_classes = ["damage"]
    metadata = MetadataCatalog.get("car_dataset")

    # Initialize predictor
    predictor = DefaultPredictor(cfg)
    MODEL_LOADED = True
except Exception as e:
    print(f"Error loading Detectron2: {e}")
    MODEL_LOADED = False

# Repair cost parameters
SEVERITY_LEVELS = {
    'Low': (1, 500),
    'Moderate': (501, 1500),
    'Severe': (1501, 3000)
}

def calculate_severity(num_damages):
    """Calculate severity based on number of damages detected"""
    if num_damages == 0:
        return "No Damage", (0, 0)
    elif num_damages <= 2:
        return "Low", SEVERITY_LEVELS['Low']
    elif 3 <= num_damages <= 4:
        return "Moderate", SEVERITY_LEVELS['Moderate']
    else:
        return "Severe", SEVERITY_LEVELS['Severe']

def generate_pdf(image_path, damage_img_path, severity, cost_estimate, damage_count):
    """Generate a detailed PDF report with the analysis results"""
    pdf_file = tempfile.NamedTemporaryFile(suffix=".pdf", delete=False)
    doc = SimpleDocTemplate(pdf_file.name, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Title
    title_style = ParagraphStyle(
        'Title',
        parent=styles['Heading1'],
        alignment=1,
        spaceAfter=14
    )
    story.append(Paragraph("Car Damage Assessment Report", title_style))

    # Company Info
    story.append(Paragraph("CarDamage Analyzer Pro", styles['Normal']))
    story.append(Paragraph("Contact: support@cardamage.ai", styles['Normal']))
    story.append(Spacer(1, 12))

    # Add current date
    story.append(Paragraph(f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}", styles['Normal']))
    story.append(Spacer(1, 24))

    # Add original image
    story.append(Paragraph("Original Image:", styles['Heading2']))
    img = ReportLabImage(image_path, width=4*inch, height=3*inch)
    story.append(img)
    story.append(Spacer(1, 12))

    # Add damage visualization image
    story.append(Paragraph("Damage Visualization:", styles['Heading2']))
    img = ReportLabImage(damage_img_path, width=4*inch, height=3*inch)
    story.append(img)
    story.append(Spacer(1, 24))

    # Damage Details
    story.append(Paragraph("Damage Assessment Summary:", styles['Heading2']))
    data = [
        ['Severity Level:', severity],
        ['Estimated Repair Cost:', cost_estimate],
        ['Number of Damages:', damage_count]
    ]
    table = Table(data, colWidths=[2*inch, 4*inch])
    table.setStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
        ('GRID', (0,0), (-1,-1), 1, colors.black)
    ])
    story.append(table)
    story.append(Spacer(1, 24))

    # Repair Steps
    repair_steps = {
        'Low': ["1. Minor surface cleaning", "2. Touch-up painting", "3. Protective coating application"],
        'Moderate': ["1. Panel straightening", "2. Medium-level repainting", "3. Component alignment"],
        'Severe': ["1. Structural frame inspection", "2. Component replacement", "3. Full repainting", "4. Mechanical system check"]
    }

    story.append(Paragraph("Recommended Repair Steps:", styles['Heading2']))
    for step in repair_steps.get(severity, ["No repairs needed"]):
        story.append(Paragraph(f"• {step}", styles['Normal']))

    story.append(Spacer(1, 24))
    story.append(Paragraph("Notes:", styles['Heading2']))
    story.append(Paragraph("• Costs are estimates based on average market rates", styles['Normal']))
    story.append(Paragraph("• Final cost may vary based on actual damage extent", styles['Normal']))

    doc.build(story)
    return pdf_file.name

def preprocess_image_for_damage(image):
    """Preprocess image to improve damage generation results"""
    img = np.array(image)

    # Enhance contrast to better identify car areas
    contrast_img = cv2.convertScaleAbs(img, alpha=1.3, beta=10)

    # Apply mild sharpening for better feature detection
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpened = cv2.filter2D(contrast_img, -1, kernel)

    # Apply subtle blur to reduce noise
    processed_img = cv2.GaussianBlur(sharpened, (3, 3), 0)

    return Image.fromarray(processed_img)

def generate_synthetic_damage(image):
    """Generate realistic synthetic damage on car images with improved targeting"""
    # First preprocess the image for better results
    processed_image = preprocess_image_for_damage(image)
    img = np.array(processed_image)
    height, width = img.shape[:2]

    # Detect car regions using color and edge information
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)

    # Analyze the image to determine the car type/view
    # Is this a rear view (like the Creta)?
    is_rear_view = False
    lower_half_mean = np.mean(img[height//2:, :])
    if lower_half_mean > 100:  # Brighter lower half often indicates rear view
        is_rear_view = True

    # Create different damage types based on the detected car view
    damages_added = 0

    # 1. Add scratches (works on any view)
    if random.random() > 0.3:
        scratch_color = (80, 80, 80)
        for _ in range(random.randint(3, 8)):
            # For rear view, focus on bumper area
            if is_rear_view:
                x1 = random.randint(width//4, 3*width//4)
                y1 = random.randint(height//2, 7*height//8)
            else:
                x1 = random.randint(width//8, 7*width//8)
                y1 = random.randint(height//4, 3*height//4)

            # Create scratch with random length and angle
            length = random.randint(20, 100)
            angle = random.uniform(0, 2*np.pi)
            x2 = int(x1 + length * np.cos(angle))
            y2 = int(y1 + length * np.sin(angle))
            thickness = random.randint(1, 3)
            cv2.line(img, (x1, y1), (x2, y2), scratch_color, thickness)
        damages_added += 1

    # 2. Add dent effect (deeper damage)
    if random.random() > 0.5:
        # Choose appropriate region for the car type
        if is_rear_view:
            # For rear view (like Creta), focus on bumper and taillight areas
            cx = random.randint(width//4, 3*width//4)
            cy = random.randint(height//2, 7*height//8)
        else:
            # For side/front views, more general positioning
            cx = random.randint(width//4, 3*width//4)
            cy = random.randint(height//3, 2*height//3)

        # Create dent effect
        radius = random.randint(30, 70)

        # Create a gradient dent effect
        for y in range(max(0, cy-radius), min(height, cy+radius)):
            for x in range(max(0, cx-radius), min(width, cx+radius)):
                distance = np.sqrt((x-cx)**2 + (y-cy)**2)
                if distance < radius:
                    # Darken the area based on distance from center
                    factor = 0.6 + 0.4 * (distance/radius)  # Darker in center
                    img[y, x] = (img[y, x] * factor).astype(np.uint8)
        damages_added += 1

    # 3. Add paint chip/discoloration
    if random.random() > 0.4:
        # Determine appropriate region
        if is_rear_view:
            # For the Creta's rear view, target the lower bumper
            x = random.randint(width//3, 2*width//3)
            y = random.randint(2*height//3, 7*height//8)
            chip_size = random.randint(15, 40)
        else:
            x = random.randint(width//4, 3*width//4)
            y = random.randint(height//4, 3*height//4)
            chip_size = random.randint(20, 50)

        # Create irregular shape for paint chip
        for dy in range(-chip_size, chip_size):
            for dx in range(-chip_size, chip_size):
                dist = np.sqrt(dx**2 + dy**2)
                if dist < chip_size and random.random() > 0.3:
                    ny, nx = y + dy, x + dx
                    if 0 <= ny < height and 0 <= nx < width:
                        # Create exposed metal or primer color
                        if random.random() > 0.5:
                            img[ny, nx] = [180, 180, 180]  # Light metal/primer
                        else:
                            img[ny, nx] = [100, 100, 100]  # Darker exposed metal
        damages_added += 1

    # 4. Add cracked taillight effect (for rear views like Creta)
    if is_rear_view and random.random() > 0.6:
        # Find taillight region (usually in upper corner quadrants for rear view)
        if random.random() > 0.5:
            # Left taillight
            tx = random.randint(width//8, width//3)
        else:
            # Right taillight
            tx = random.randint(2*width//3, 7*width//8)

        ty = random.randint(height//3, height//2)

        # Create crack pattern in taillight
        crack_color = (255, 255, 255)  # White crack
        start_x, start_y = tx, ty

        for _ in range(random.randint(3, 8)):
            length = random.randint(10, 30)
            angle = random.uniform(0, 2*np.pi)
            end_x = int(start_x + length * np.cos(angle))
            end_y = int(start_y + length * np.sin(angle))

            cv2.line(img, (start_x, start_y), (end_x, end_y), crack_color, 1)
            start_x, start_y = end_x, end_y
        damages_added += 1

    # Apply overall filter to blend damages
    result_img = cv2.GaussianBlur(img, (3, 3), 0)

    return Image.fromarray(result_img), damages_added

def predict_damage(image):
    """Main function to detect car damage in the given image"""
    if not MODEL_LOADED:
        return "Error: Model could not be loaded. Please check your Detectron2 installation."

    if image is None:
        return None, "No Damage", "No damage detected", "0", None

    # Convert PIL Image to OpenCV format
    img = np.array(image)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Run inference
    outputs = predictor(img_bgr)
    instances = outputs["instances"].to("cpu")

    # Process detections
    num_damages = len(instances)

    # Calculate severity and cost
    severity_level, cost_range = calculate_severity(num_damages)
    cost_estimate = f"₹{cost_range[0]} - ₹{cost_range[1]}" if num_damages > 0 else "No damage detected"

    # Visualize results
    v = Visualizer(img_bgr[:, :, ::-1], metadata=metadata, scale=0.8)
    out = v.draw_instance_predictions(instances)
    result_img = Image.fromarray(out.get_image()[:, :, ::-1])

    # Save images for the PDF report
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_orig_img, \
         tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_result_img:
        # Save original image
        image.save(temp_orig_img.name, format='JPEG')
        # Save result image
        result_img.save(temp_result_img.name, format='JPEG')

        # Generate PDF report
        pdf_path = generate_pdf(
            temp_orig_img.name,
            temp_result_img.name,
            severity_level,
            cost_estimate,
            str(num_damages)
        )

    return result_img, severity_level, cost_estimate, str(num_damages), pdf_path

def synthetic_damage_demo(image):
    """Demo function to generate and show synthetic damage on car images"""
    if image is None:
        return None

    # Apply synthetic damage with enhanced algorithm
    damaged_image, _ = generate_synthetic_damage(image)
    return damaged_image

# Create Gradio interface
with gr.Blocks(title="Car Damage Assessment System") as demo:
    gr.Markdown("# Car Damage Assessment System")
    gr.Markdown("Upload an image of a damaged car to assess severity, repair cost, and view detected damages")

    with gr.Tabs():
        with gr.TabItem("Damage Assessment"):
            with gr.Row():
                with gr.Column():
                    input_image = gr.Image(label="Upload Car Image", type="pil")
                    assess_button = gr.Button("Assess Damage", variant="primary")

                with gr.Column():
                    output_image = gr.Image(label="Damage Visualization")
                    severity_output = gr.Textbox(label="Damage Severity")
                    cost_output = gr.Textbox(label="Estimated Repair Cost")
                    count_output = gr.Textbox(label="Damage Count")
                    report_output = gr.File(label="Download Report")

            assess_button.click(
                fn=predict_damage,
                inputs=input_image,
                outputs=[output_image, severity_output, cost_output, count_output, report_output]
            )

        with gr.TabItem("Synthetic Damage Generator"):
            with gr.Row():
                with gr.Column():
                    synth_input = gr.Image(label="Upload Clean Car Image", type="pil")
                    synth_button = gr.Button("Generate Synthetic Damage", variant="secondary")

                with gr.Column():
                    synth_output = gr.Image(label="Synthetic Damaged Image")

            synth_button.click(
                fn=synthetic_damage_demo,
                inputs=synth_input,
                outputs=synth_output
            )

    gr.Markdown("## How it works")
    gr.Markdown("""
    This system uses computer vision techniques to:
    1. Detect damage areas on vehicle images
    2. Analyze damage severity and estimated repair costs
    3. Generate comprehensive repair reports
    4. Create realistic synthetic damage for testing and training

    The enhanced synthetic damage generator can work with different car views,
    including challenging rear views like the Hyundai Creta.
    """)

    # Add examples if they exist
    example_dir = "examples" if os.path.exists("examples") else "."
    example_images = [
        os.path.join(example_dir, "sample_damage1.jpg"),
        os.path.join(example_dir, "sample_damage2.jpg")
    ]

    if any(os.path.exists(p) for p in example_images):
        gr.Examples(
            examples=[[p] for p in example_images if os.path.exists(p)],
            inputs=input_image
        )

# Launch the application
if __name__ == "__main__":
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b9553b4171fdee1a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
